In [1]:
import numpy as np
import math

In [2]:
# GLOBAL VARIABLES
# text of 80 chars -> 640 bits
# we encode encode each bit into 51200/640 = 80 bits

N = 40960 #max is 51200, 40960 = 64*640
TOTAL_NB_CODEWORDS = 640 
CODEWORD_SIZE = 64 # 64 is the closest power 2 to 80

# with CODEWORD_SIZE = 2^i
i = 6

# M is CODEWORD_SIZE * CODEWORD_SIZE, we want sqrt(EPSILON/CODEWORD_SIZE) = 1
EPSILON = CODEWORD_SIZE

In [3]:
def compute_M(j):
    if j== 0:
        return np.array([1])
    M_1 = np.array([[1,1],[1,-1]])
    temp = M_1
    for l in range (j-1):
        up = np.concatenate((temp,temp), axis=1) 
        bottom = np.concatenate((temp,-temp), axis=1)
        temp = np.concatenate((up,bottom), axis=0)
    return temp

In [4]:
M=compute_M(2)
print(M)


[[ 1  1  1  1]
 [ 1 -1  1 -1]
 [ 1  1 -1 -1]
 [ 1 -1 -1  1]]


In [5]:
def columnk_M(M,k):
    if M.shape == (1,):
        return M[k-1]
    return M[:,k-1]


In [6]:
columnk_M(M,2)

array([ 1, -1,  1, -1])

In [8]:
def encode_mk(M,k,sign):
    return (sign*columnk_M(M,k))

In [9]:
M2=compute_M(0)
print(M2.shape)

(1,)


In [11]:
print(encode_mk(M,3,-1))

[-1 -1  1  1]


In [15]:
def compute_U(M,Y,j):
    if j==0:
        return Y
    if j==1:
        return np.dot(M,Y)
    U_bis = compute_M(j-1)
    U_t = np.dot(U_bis,Y[0:int(CODEWORD_SIZE/2)])
    U_b = np.dot(U_bis,Y[int(CODEWORD_SIZE/2):])
    return np.concatenate((U_t+U_b,U_t-U_b),axis=0)   

In [17]:
M1=compute_M(i)
U1=compute_U(M1,encode_mk(M1,1,-1),i)

In [18]:
def decode_U(U):
    U_bis=abs(U)
    k=np.argmax(U_bis)
    if U[int(k)]<0:
        return -(k+1)
    else :
        return k+1

In [19]:
decode_U(U1)

-1

In [21]:
def text_to_bits(textfile): # we encode each text bit into CODEWORD_SIZE bits
    f=open(textfile,'rb')
    data = f.read()
    f.close()
    #noOfBytes = len(data)
    #noOfChunks= noOfBytes/CHUNK_SIZE
    #if(noOfBytes%CHUNK_SIZE):
    #    noOfChunks+=1
        
    textBitVector = []
    for i in range(0, TOTAL_NB_CODEWORDS+1, CODEWORD_SIZE):
        np.append(textBitVector,data[i:i+ CODEWORD_SIZE])
    
    return textVector

In [ ]:
def chunks_to_text(fileName,chunkVector):
    f = open(fileName, 'wb')
    for chunk in chunkVector:
        f.write(chunk)
        f.close()
    return fileName   

In [114]:
def chunk_to_codewords(chunk):
    i = CHUNK_SIZE-1
    n = np.power(2,i)
    M = compute_M(i)
    chunkValue = chunk.dot(2**np.arange(chunkValue.size)[::-1])
    k = chunkValue%2
    if ((chunkValue/2) > n):
        sign = -1
    else :
        sign = 1
    return encode_mk(M,k,epsilon,n,sign)       

In [12]:
def transmitter(filename):
    print(len(filename)*8)
    textVector = text_to_chunks(filename)
    message = []
    for i in range (len(chunks)):
        np.append(message,chunk_to_codewords(textVector[i],CODEWORD_SIZE,n))
    return np.savetxt("message.txt", message, fmt="%s")
    

In [13]:
def receiver(filename):
    n=np.power(2,15)
    i = math.log2(n)
    CHUNK_SIZE = math.ceil(n/(len(filename)*8))
    chunks = text_to_chunks(filename,CHUNK_SIZE)
    M = compute_M(i)
    message = []
    for j in range (len(chunks)):
        U = compute_U(M,chunk[j],i)
        np.append(message,decode_U(U,j))
    return np.savetxt("received.txt", message, fmt="%s")
    
        

In [15]:
transmitter("C:/Users/julie/PDCproj1.txt")

216
152
